In [1]:
import ROOT
import numpy as np

OBJ: TStyle	ildStyle	ILD Style : 0 at: 0x6799860


In [2]:
%jsroot on

In [3]:
# FIXME: don't touch! the fortran code has a global state :(
# ROOT.EnableImplicitMT(6)
ROOT.TH1.SetDefaultSumw2()

In [4]:
ROOT.gSystem.Load("OO/diehl/libObservable.so")

0

In [5]:
ROOT.gInterpreter.Declare("#include \"OO/diehl/OOTools.h\"")

True

In [6]:
# use same masses as in whizard!
mW = 80.419
mZ = 91.1882

ROOT.OOTools.init(250, mW, mZ)

In [7]:
ROOT.ildStyle.SetOptStat(1)
# ROOT.ildStyle.SetOptStat(0)

In [8]:
%%cpp
using namespace ROOT::VecOps;

In [9]:
# n_points = 5000000
n_points = 1000000

df = ROOT.RDataFrame(n_points)

In [10]:
df = df.Define("co", "gRandom->Uniform(-1., 1.)")
df = df.Define("co1", "gRandom->Uniform(-1., 1.)")
df = df.Define("co2", "gRandom->Uniform(-1., 1.)")
df = df.Define("ph1", "gRandom->Uniform(-1.*ROOT::Math::Pi(), 1.*ROOT::Math::Pi())")
df = df.Define("ph2", "gRandom->Uniform(-1.*ROOT::Math::Pi(), 1.*ROOT::Math::Pi())")

In [11]:
df = df.Define("w_proba", "OOTools::proba(co, co1, co2, ph1, ph2, 0, true, 0)")
df = df.Define("w_proba_fz1", "OOTools::proba(co, co1, co2, ph1, ph2, 1, false, 1)")

In [12]:
for var in [f"fg{i}" for i in range(1,8)] + [f"fz{i}" for i in range(1,8)]:
    df = df.Define(var, f"OOTools::{var}(co, co1, co2, ph1, ph2)")

In [13]:
f_vars = ["fg1", "fz1", "fg2", "fz2", "fg3", "fz3", "fg5", "fz5"]
columns = df.AsNumpy(f_vars+ ["w_proba", "w_proba_fz1"])

In [14]:
f_vars_a = np.asarray([columns[v] for v in f_vars])

In [15]:
# w_proba_norm = columns["w_proba"] / np.max(columns["w_proba"])
# w_proba_fz1_norm = columns["w_proba_fz1"] / np.max(columns["w_proba"])
# normalized such that we have frequencies for the SM values
w_proba_norm = (columns["w_proba"] / np.min(columns["w_proba"])).astype(int)
w_proba_fz1_norm = columns["w_proba_fz1"] / np.min(columns["w_proba"])

In [16]:
# ugh aweights and fweights are not equivalent :(
# f_vars_cov = np.cov(f_vars_a, ddof=0, aweights=w_proba_norm)
f_vars_cov = np.cov(f_vars_a, ddof=0, fweights=w_proba_norm)
print(np.round(1.e3 * f_vars_cov, decimals=2))
# meh this c is wrong :(
# even when going to fweights :(

[[ 120.78  100.78 -421.23 -331.21 -186.31 -126.7     6.86   -2.94]
 [ 100.78  131.92 -339.01 -442.81 -143.28 -188.79    3.35    3.21]
 [-421.23 -339.01 1551.78 1223.44  684.43  463.41  -26.31   17.97]
 [-331.21 -442.81 1223.44 1671.23  509.35  713.57  -16.66   -9.61]
 [-186.31 -143.28  684.43  509.35  365.22  250.73  -15.95   28.2 ]
 [-126.7  -188.79  463.41  713.57  250.73  404.24  -12.17   -8.63]
 [   6.86    3.35  -26.31  -16.66  -15.95  -12.17   34.72   42.33]
 [  -2.94    3.21   17.97   -9.61   28.2    -8.63   42.33   93.74]]


In [17]:
diffs = [0.] * len(f_vars)

for i, f in enumerate(f_vars):
    mean_1 = np.average(columns[f], weights=0.1*w_proba_fz1_norm)
    mean_0 = np.average(columns[f], weights=w_proba_norm)
    diffs[i] = mean_1 - mean_0

g_vec = np.linalg.solve(f_vars_cov, np.asarray(diffs))
print(g_vec)

[-1.28374284e-01  2.03934025e+01 -1.72857663e-02  1.06969686e-03
 -3.18423393e-02  3.00643532e-02 -4.71681383e-02  2.35709821e-02]


In [18]:
pi = ROOT.Math.Pi()

# or just fill 1D histograms with weight proba??
# p_co = df.Profile1D(("", ";co", 25, -1., 1.), "co", "proba")
# p_co1 = df.Profile1D(("", ";co1", 25, -1., 1.), "co1", "proba")
# p_co2 = df.Profile1D(("", ";co2", 25, -1., 1.), "co2", "proba")
# p_ph1 = df.Profile1D(("", ";ph1", 25, -pi, pi), "ph1", "proba")
# p_ph2 = df.Profile1D(("", ";ph2", 25, -pi, pi), "ph2", "proba")
h_co  = df.Histo1D(("", ";co", 25, -1., 1.), "co", "w_proba")
h_co1 = df.Histo1D(("", ";co1", 25, -1., 1.), "co1", "w_proba")
h_co2 = df.Histo1D(("", ";co2", 25, -1., 1.), "co2", "w_proba")
h_ph1 = df.Histo1D(("", ";ph1", 25, -pi, pi), "ph1", "w_proba")
h_ph2 = df.Histo1D(("", ";ph2", 25, -pi, pi), "ph2", "w_proba")

h_weight = df.Histo1D("w_proba")

h_fg1 = df.Histo1D(("", ";fg1", 100, -1.5, 1.5), "fg1", "w_proba")
h_fg2 = df.Histo1D(("", ";fg2", 100, -1.5, 1.5), "fg2", "w_proba")
h_fg3 = df.Histo1D(("", ";fg3", 100, -1.5, 1.5), "fg3", "w_proba")
h_fg4 = df.Histo1D(("", ";fg4", 100, -1.5, 1.5), "fg4", "w_proba")
h_fg5 = df.Histo1D(("", ";fg5", 100, -1.5, 1.5), "fg5", "w_proba")

h_fz1 = df.Histo1D(("", ";fz1", 100, -1.5, 1.5), "fg1", "w_proba")
h_fz2 = df.Histo1D(("", ";fz2", 100, -1.5, 1.5), "fg2", "w_proba")
h_fz3 = df.Histo1D(("", ";fz3", 100, -1.5, 1.5), "fg3", "w_proba")
h_fz4 = df.Histo1D(("", ";fz4", 100, -1.5, 1.5), "fg4", "w_proba")
h_fz5 = df.Histo1D(("", ";fz5", 100, -1.5, 1.5), "fg5", "w_proba")

In [19]:
c_co = ROOT.TCanvas()
# p_co.SetMinimum(0)
# p_co.Draw()
h_co.SetMinimum(0)
h_co.Draw()
c_co.Draw()

c_co1 = ROOT.TCanvas()
# p_co1.SetMinimum(0)
# p_co1.Draw()
h_co1.SetMinimum(0)
h_co1.Draw()
c_co1.Draw()

c_ph1 = ROOT.TCanvas()
# p_ph1.SetMinimum(0)
# p_ph1.Draw()
h_ph1.SetMinimum(0)
h_ph1.Draw()
c_ph1.Draw()

c_co2 = ROOT.TCanvas()
# p_co2.SetMinimum(0)
# p_co2.Draw()
h_co2.SetMinimum(0)
h_co2.Draw()
c_co2.Draw()

c_ph2 = ROOT.TCanvas()
# p_ph2.SetMinimum(0)
# p_ph2.Draw()
h_ph2.SetMinimum(0)
h_ph2.Draw()
c_ph2.Draw()

c_weight = ROOT.TCanvas()
h_weight.SetMinimum(0)
h_weight.Draw()
c_weight.Draw()


In [21]:

c_fg1 = ROOT.TCanvas()
h_fg1.SetMinimum(0)
h_fg1.Draw()
c_fg1.Draw()

c_fg2 = ROOT.TCanvas()
h_fg2.SetMinimum(0)
h_fg2.Draw()
c_fg2.Draw()

c_fg3 = ROOT.TCanvas()
h_fg3.SetMinimum(0)
h_fg3.Draw()
c_fg3.Draw()

c_fg4 = ROOT.TCanvas()
h_fg4.SetMinimum(0)
h_fg4.Draw()
c_fg4.Draw()

c_fg5 = ROOT.TCanvas()
h_fg5.SetMinimum(0)
h_fg5.Draw()
c_fg5.Draw()

c_fz1 = ROOT.TCanvas()
h_fz1.SetMinimum(0)
h_fz1.Draw()
c_fz1.Draw()

c_fz2 = ROOT.TCanvas()
h_fz2.SetMinimum(0)
h_fz2.Draw()
c_fz2.Draw()

c_fz3 = ROOT.TCanvas()
h_fz3.SetMinimum(0)
h_fz3.Draw()
c_fz3.Draw()

c_fz4 = ROOT.TCanvas()
h_fz4.SetMinimum(0)
h_fz4.Draw()
c_fz4.Draw()

c_fz5 = ROOT.TCanvas()
h_fz5.SetMinimum(0)
h_fz5.Draw()
c_fz5.Draw()